### Used Libraries

In [ ]:
from IPython.display import clear_output 
import plotly
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from scipy import stats, signal
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
from scipy.signal import savgol_filter, general_gaussian
from scipy.signal import detrend as scipydetrend
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn import linear_model
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import f, chi2, norm, sem
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.model_selection import train_test_split, LeaveOneOut, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from obspy.signal import detrend as obspydetrend
from verstack.stratified_continuous_split import scsplit
import os
import statistics

### Classes and Functions

In [ ]:
class ScatterCorrection():

  def Normalisation(input_data):

    if 'sample' in input_data.columns:
      start=1
    else:
      start=0

    data_normal=StandardScaler().fit_transform(input_data.iloc[:,start:])
    data_normal=pd.DataFrame(data_normal, index=input_data['sample'] if 'sample' in input_data.columns else input_data.index, columns=input_data.columns[start:])

    return (data_normal)  

  def SNV(input_data):
      ''' Perform Standard Normal Variate scatter correction.

      There are two methods to be choosen (1) MSC and (2) SNV. 
      For SNV, the only function parameter is data frame which needs to be defined.
      For MSC, the user can choose some options apart from data frame as:
      - method='MSC' (default), 'IMSC' 
      - meancentering= None (default), 'all', 'group' 
      - reference= a external reference can be loaded. Otherwise mean of the data set will be used ('mean').'''

      #clean '.*' extensions of sample names
      #for i in range(len(input_data['sample'])):
       # input_data['sample'][i]=input_data['sample'][i][:-6]    
      #input_data=input_data.set_index('sample')
      
      # Define a new array and populate it with the corrected data    
      data_snv = pd.DataFrame().reindex_like(input_data)
      if 'sample' in input_data.columns:
        data_snv['sample']=input_data['sample']
        data_snv=data_snv.set_index('sample')
        start=1
      else:
        start=0

      for i in range(input_data.shape[0]):
        # Apply correction
        data_snv.iloc[i,:] = (input_data.iloc[i,start:] - np.mean(input_data.iloc[i,start:])) / np.std(input_data.iloc[i,start:])
      
      #input_data=input_data.set_index(input_data.index)
      return (data_snv)  

  def MSC(input_data, method='MSC', meancentering='all', reference='mean'):
      ''' Perform Multiplicative scatter correction
      
      There are two methods to be choosen (1) MSC and (2) SNV. 
      For SNV, the only function parameter is data frame which needs to be defined.
      For MSC, the user can choose some options apart from data frame as:
      - method='MSC' (default), 'IMSC' 
      - meancentering= None (default), 'all', 'group' 
      - reference= a external reference can be loaded. Otherwise mean of the data set will be used 'mean' or 'median'.'''

      #clean '.*' extensions of sample names
      #for i in range(len(input_data['sample'])):
        #input_data['sample'][i]=input_data['sample'][i][:-6]
      
      # mean centre correction is none
      if meancentering==None:
        input_data=input_data.set_index('sample')
      # mean centre correction according to the sample group label
      elif meancentering=='group':      
        df_mean=input_data.groupby(['sample']).mean()
        input_data=input_data.set_index('sample')
        for name in df_mean.index:
          input_data.loc[name]=input_data.loc[name]-df_mean.loc[name]
      # mean centre correction using all spectra irrespective of sample groups
      elif meancentering=='all':
        if 'sample' in input_data.columns:
          input_data=input_data.set_index('sample')
        if reference == 'mean':
          for i in range(input_data.shape[0]):
            input_data.iloc[i,:] -= input_data.iloc[i,:].mean()
        elif reference == 'median':
          for i in range(input_data.shape[0]):
            input_data.iloc[i,:] -= input_data.iloc[i,:].median()

      # Get the reference spectrum. If not given, estimate it from the mean   
      if reference == 'mean':    
          # Calculate mean
          ref = input_data.mean()
      elif reference == 'median': 
          ref = input_data.median()
      else:
          ref = reference

      # Define a new array and populate it with the corrected data    
      data_msc = pd.DataFrame().reindex_like(input_data)
      if 'sample' in input_data.columns:
        data_msc['sample']=input_data['sample']
        data_msc=data_msc.set_index('sample')
        start=1
      else:
        start=0

      for i in range(input_data.shape[0]):
        # Run regression
        X = ref.values.reshape(-1, 1)  # values converts it into a numpy array
        y = input_data.iloc[1,start:].values.reshape(-1, 1)
        linear_regressor = LinearRegression()  # create object for the class
        if method=='MSC': 
          linear_regressor.fit(X=X, y=y)  # perform linear regression for MSC
        elif method=='IMSC': 
          linear_regressor.fit(X=y, y=X)  # perform linear regression for Inverted MSC
          # Apply correction
        data_msc.iloc[i,:] = (input_data.iloc[i,start:] - linear_regressor.intercept_[0]) / linear_regressor.coef_[0]
      
      return (data_msc) 
  
  def MinMaxNorm(input_data):
    # copy the data
    df_min_max_scaled = input_data.copy()
  
    # apply normalization techniques
    if 'sample' in input_data.columns:
      for column in df_min_max_scaled.columns[df_min_max_scaled.columns != 'sample']:
        df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())
    else: 
      for column in df_min_max_scaled.columns:
        df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())
    
    return (df_min_max_scaled)




class SmoothandDeriv():
  def RemoveSpike(input_data, kernelsize):
    despiked=input_data.copy()

    if 'sample' in input_data.columns:
      despiked['sample']=input_data['sample']
      despiked=despiked.set_index('sample')
      start=1
    else:
      start=0

    for i in range(0, input_data.shape[0]):
      despiked.iloc[i,start:] = signal.medfilt(input_data.iloc[i,start:], kernelsize)

    return (despiked)

  def Derivative(input_data,method='central'):
      '''Compute the first derivative of the spectras.

      Returns
      -------
      float
          Difference formula:
              central: f(a+h) - f(a-h))/2h
              forward: f(a+h) - f(a))/h
              backward: f(a) - f(a-h))/h            
      '''

      if 'sample' not in input_data.columns:
        input_data.insert(0, 'sample', input_data.index)

      df = pd.DataFrame().reindex_like(input_data)
      df.iloc[:,:]=input_data.iloc[:,:]

      #global df_derivates
      df_derivates = pd.DataFrame(columns=df.columns[:-1])
      df_derivates['sample']=df['sample']
  
      if method == 'central':
        for row in range(len(df['sample'])):
          for col in range(len(df.columns[1:])-2):
            df_derivates.iloc[row,col+1]=(df.iloc[row, col+3]-df.iloc[row, col+1])/(float(df.columns[col+3])-float(df.columns[col+1]))
      elif method == 'forward':
        for row in range(len(df['sample'])):
          for col in range(len(df.columns[1:])-1):
            df_derivates.iloc[row,col+1]=(df.iloc[row, col+2]-df.iloc[row, col+1])/(float(df.columns[col+2])-float(df.columns[col+1]))
      elif method == 'backward':
        for row in range(len(df['sample'])):
          for col in range(len(df.columns[1:])-1):
            df_derivates.iloc[row,col+1]=(df.iloc[row, col+1]-df.iloc[row, col+2])/(float(df.columns[col+2])-float(df.columns[col+1]))
      else:
          raise ValueError("Method must be 'central', 'forward' or 'backward'.")
      
      return (df_derivates.dropna(axis=1, how='all', )) 
  
  def SG(input_data, window=5, polyorder=2, derivorder=1):
    '''Applies Savitzky-Golay Filter to the given data.
    - window: window length is similar moving window smoothing but this is the number of data to be fitted to polynomial func. (give an odd number)
    - polyorder: order of polynomil function which the data is fitted.
    - derivorder: it is the degree/order or numerical derivation after smoothing.'''
    
    smoothdata = pd.DataFrame().reindex_like(input_data)

    if 'sample' in input_data.columns:
      smoothdata['sample']=input_data['sample']
      smoothdata=smoothdata.set_index('sample')
      start=1
    else:
      start=0

    for i in range(0,data.shape[0],1):
      smoothdata.iloc[i,0:] = savgol_filter(input_data.iloc[i,start:], window_length=window, polyorder = polyorder, deriv=derivorder)
    
    return (smoothdata)

  def NW(input_data, window=3, derivorder=1):  
    '''Applies Norris-Williams Filter to the given data.
    - window: window length is similar moving window smoothing but this is the number of data to be included in smooting and derivation. (give an odd number)
    - derivorder: it is the degree/order or numerical derivation after smoothing.'''    
    if (window % 2) == 0:
      return(print ('Window size needs to be an odd number!'))

    smoothdata = pd.DataFrame().reindex_like(input_data)
    if 'sample' in input_data.columns:
      smoothdata['sample']=input_data['sample']
      smoothdata=smoothdata.set_index('sample')
      start=1
    else:
      start=0
    
    # the first step (smoothing) of the norris-williams derivation
    for k in range(0,input_data.shape[0],1):
      i = 0
      print('smoothing sample-'+str(k))
      while i < input_data.shape[1] - window + 1:
          smoothdata.iloc[k,i+math.floor(window/2)] = sum(input_data.iloc[k, i+start : i+start+ window -1]) / (window+1)
          i += 1
    smoothdata = smoothdata.iloc[:,math.floor(window/2):-math.floor(window/2)]

    # the second step (derivation) of the norris-williams derivation
    derivdata = pd.DataFrame().reindex_like(smoothdata)
    for k in range(0,smoothdata.shape[0],1):
      i = 0
      print('derivating sample-'+str(k))
      if derivorder == 1:
        while i < smoothdata.shape[1] - window + 1:
          derivdata.iloc[k,i+math.floor(window/2)] = (smoothdata.iloc[k, i] - smoothdata.iloc[k, i+window-1])
          i += 1
        #return (smoothdata.iloc[:,math.floor(window/2):-math.floor(window/2)])
      if derivorder == 2:
        while i < smoothdata.shape[1] - window + 1:
          derivdata.iloc[k,i+math.floor(window/2)] = (smoothdata.iloc[k, i+start] - 2*smoothdata.iloc[k, i+math.floor(window/2)] + smoothdata.iloc[k, i+window-1])
          i += 1
        #return (smoothdata.iloc[:,2*math.floor(window/2):-2*math.floor(window/2)]) 
    return (derivdata.iloc[:,math.floor(window/2):-math.floor(window/2)])    


def deTrend(input_data, method='linregres', order=None, spline_gap=None, plot=False):
  '''Applies detrending to the given data.
  - method: 'linregres', fits a linear func to the spectrum and removes the signal from the regression line.
  'meansubstract', mean centers the data using th mean af a spectra.
  'highorder', fits a >= 2nd order polynomial funstion to the data and takes the diffrecence. if the order is 1, 'highorder' and 'linregres' is almost identical.
  'simple', draws a line between first and last points of the spectrum and takes the diference of this line and the signal.
  'spline', fits multpline polynomial fnctions for each spline_gap number of data with the given order and takes the difference.
  - order: the order of fitted polynomial function for highorder and spline methods.
  - plot: True or False only for for highorder and spline methods. '''
  
  detrenddata = pd.DataFrame().reindex_like(input_data)
  if 'sample' in input_data.columns:
    detrenddata['sample']=input_data['sample']
    detrenddata=detrenddata.set_index('sample')
    start=1
  else:
    start=0
  
  if method=='linregres':
    for i in range(0,data.shape[0],1):
      detrenddata.iloc[i,0:] = scipydetrend(input_data.iloc[i,start:], type='linear')
  elif method=='meansubstract':
    for i in range(0,data.shape[0],1):
       detrenddata.iloc[i,0:] = scipydetrend(input_data.iloc[i,start:], type='constant')
  elif method=='highorder':
    for i in range(0,data.shape[0],1):
      detrenddata.iloc[i,0:] = obspydetrend.polynomial(input_data.iloc[i,start:], order=order, plot=plot if i==data.shape[0]-1 else False)
  elif method=='simple':
    for i in range(0,data.shape[0],1):
      detrenddata.iloc[i,0:] = obspydetrend.simple(input_data.iloc[i,start:])
  elif method=='spline':
    for i in range(0,data.shape[0],1):
      detrenddata.iloc[i,0:] = obspydetrend.spline(data.iloc[0,1:], order=order, dspline=spline_gap, plot=plot if i==data.shape[0]-1 else False) 

  return (detrenddata)



def RMS_repeatability(input_data, central='mean', conf=0.95, multiplicator=1e6, reducename=False, reducechar=6):
   "this function calculates RMS, STDlimit and lists samples to be repeated"

   df = pd.DataFrame().reindex_like(input_data)
   df.iloc[:,:]=input_data.iloc[:,:]

   global results
   if reducename==True:
     for i in range(len(df['sample'])):
       df['sample'][i]=df['sample'][i][:-reducechar]

   if central =='mean':
     df_mean=df.groupby(['sample']).mean()
   elif central =='median':
     df_mean=df.groupby(['sample']).median()
   df=df.set_index('sample')
 
   df_subs=df
   for name in df_mean.index:
     df_subs.loc[name]=df.loc[name]-df_mean.loc[name]
  
   df['RMS']=multiplicator*np.sqrt(np.square(df_subs).mean(axis='columns'))
   RMS_mean=np.sqrt(np.square(df).groupby(by=['sample']).mean())['RMS']

   c = Counter(df.index) 
   STD=RMS_mean
   for name in RMS_mean.index:
     STD.loc[name]=RMS_mean.loc[name]*np.sqrt(c[name]/(c[name]-1))
  
   STD_limit=np.sqrt(np.square(STD).mean(axis='rows'))* stats.t.ppf(conf, len(STD))

   results=pd.DataFrame(columns=['RMS', 'STDLimit','Acceptability'], index=df.index)
   results['RMS']=round(df['RMS'],2)
   results['STDLimit']=round(STD_limit,2)
   results['Acceptability']=results['RMS']<STD_limit
   booleanDictionary = {True: 'Accept', False: '* Repeat'}
   results['Acceptability'] = results['Acceptability'] .replace(booleanDictionary)
   pd.set_option('display.max_rows', None)
   
   return(results)




def RMS_graph(results):    
   "this function creates a graph to visualise RMS, STDlimit and samples to be repeated"
   
   fig1 = px.scatter(y=results['RMS'],  color=results['Acceptability'], symbol = results['Acceptability'] )
   fig2 = px.line( y=results['STDLimit'])    
    
   fig2.update_traces(line_color='firebrick')
   fig3 = go.Figure(data=fig1.data + fig2.data)
   fig3.update_traces(marker_size=5, showlegend=False)
   fig3.update_layout(
       title="NIR Repetability Test Results",
       yaxis_title="<b>Root Mean Square (x10)</b>",
       xaxis_title="<b>Tea Samples</b>",
       xaxis=dict(
           showgrid=True,
           showline=True,
           showticklabels=False,
           ticks='outside',
           tickcolor='rgb(102, 102, 102)',
           linecolor='rgb(102, 102, 102)',
       ),       
       yaxis=dict(
           showgrid=True,
           showline=True,
           showticklabels=True,
           ticks='outside',
           tickcolor='rgb(102, 102, 102)',
           linecolor='rgb(102, 102, 102)',
       ),
       #font=dict(size=5),
       margin=dict(l=40, r=40, b=40, t=40),
       paper_bgcolor='white',
       plot_bgcolor='white',
       width=300, height=300,
   )

   fig3.show()
   return(fig3)






def DrawSpect(input_data):
  fig = go.Figure()

  if 'sample' in input_data.columns:
    names=input_data['sample'].values
    start=1
  else:
    names=input_data.index
    start=0

  for i in range(0,input_data.shape[0],1):
    fig.add_trace(go.Scatter(x=input_data.columns[start:].values.astype(float), y=input_data.iloc[i,start:], mode='lines', name=str(names[i])))
 
  fig.update_traces(marker_size=3)
  fig.update_layout(
      title="NIR Spectras",
      yaxis_title="<b>log(1/R)</b>",
      xaxis_title="<b>Wavelength (nm)</b>",
      xaxis=dict(
          showgrid=True,
          showline=True,
          showticklabels=True,
          ticks='outside',
          tickcolor='rgb(102, 102, 102)',
          linecolor='rgb(102, 102, 102)',
          tickmode = 'linear',
          #tick0 = 0.5,
          dtick = 200,
      ),
      yaxis=dict(
          showgrid=True,
          showline=True,
          showticklabels=True,
          ticks='outside',
          tickcolor='rgb(102, 102, 102)',
          linecolor='rgb(102, 102, 102)',
          #tickmode = 'linear',
          #tick0 = 0.5,
          #dtick = 0.1
      ),    
      margin=dict(l=40, r=40, b=40, t=40),
      paper_bgcolor='white',
      plot_bgcolor='white',
      width=600, height=400,
  ) 
 
  fig.show()
  return(fig)



class PCR:

  def Outliers(X, y, pc, criteria='chisq', conf=0.95):
    ''' criteria: 'chisq','zdist', '3std', '4std', 'average' '''
    #pc=5
    # Define the PCA object and fit transform
    pca = PCA()
    T = pca.fit_transform(X)

    # fit a Minimum Covariance Determinant (MCD) robust estimator to data 
    robust_cov = MinCovDet().fit(T[:,:pc])
 
    # Get the Mahalanobis distance
    mahala = robust_cov.mahalanobis(T[:,:pc])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=T[:, 0], y=T[:, 1], mode='markers+text',
                          name='R2', marker_color=mahala, text=np.round(mahala[:],2) , textposition="top center"))
    fig.update_layout(
          title="PCA Score Plot and Mahalanobis distance",
          yaxis_title= "PC2 (" + str(np.round(pca.explained_variance_ratio_[1]*100,2)) +"%)" ,
          xaxis_title="PC1 (" + str(np.round(pca.explained_variance_ratio_[0]*100,2)) +"%)" ,
          xaxis=dict(
              showgrid=True,
              showline=True,
              showticklabels=True,
              ticks='outside',
              tickcolor='rgb(102, 102, 102)',
              linecolor='rgb(102, 102, 102)',
          ),
          yaxis=dict(
              showgrid=True,
              showline=True,
              showticklabels=True,
              ticks='outside',
              tickcolor='rgb(102, 102, 102)',
              linecolor='rgb(102, 102, 102)',
          ),    
          margin=dict(l=140, r=40, b=50, t=80),
          paper_bgcolor='white',
          plot_bgcolor='white',
          width=400, height=400,
        )
    fig.show()

    fig_mahala = go.Figure()
    mahala_name_criteria=np.full((len(mahala)), mahala.mean()+sem(mahala)*norm.ppf(conf))
    fig_mahala.add_trace(go.Scatter(y=mahala, x=np.linspace(1, len(mahala)), mode='markers+text', marker_line_width=2, 
                                    text=np.where(mahala_name_criteria>mahala,'',X.index), textposition="middle right",
                                    marker=dict(color=mahala)))  
    fig_mahala.update_traces(showlegend=False)
    if criteria=='average':
        fig_mahala.add_trace(go.Scatter(y=[mahala.mean(), mahala.mean()], x=[1, len(mahala)], mode='lines', name='Average Mahalanobis Distance',
                                    line=dict(color='red', width=2, dash='dash')))
    if criteria=='zdist':
        fig_mahala.add_trace(go.Scatter(y=[mahala.mean()+sem(mahala)*norm.ppf(conf), mahala.mean()+sem(mahala)*norm.ppf(conf)],
                                    x=[1, len(mahala)], mode='lines', name=str(conf) + ' Probability (z-dist)',
                                    line=dict(color='red', width=2, dash='dash')))
    if criteria=='chisq':
        fig_mahala.add_trace(go.Scatter(y=[chi2.ppf((1-conf), df=len(mahala)-1,), chi2.ppf((1-conf), df=len(mahala)-1,)],
                                    x=[1, len(mahala)], mode='lines', name=str(conf) + ' Probability (chi<sup>2</sup> dist)',
                                    line=dict(color='red', width=2, dash='dash')))
    if criteria=='3std':
        fig_mahala.add_trace(go.Scatter(y=[mahala.mean()+3*mahala.std(), mahala.mean()+3*mahala.std()], 
                                    x=[1, len(mahala)], mode='lines', name=str(conf) + ' Probability (+3 std)',
                                    line=dict(color='red', width=2, dash='dash')))
    if criteria=='4std':
        fig_mahala.add_trace(go.Scatter(y=[mahala.mean()+4*mahala.std(), mahala.mean()+4*mahala.std()], 
                                    x=[1, len(mahala)], mode='lines', name=str(conf) + ' Probability (+4 std)',
                                    line=dict(color='red',  width=2, dash='dash')))
    fig_mahala.update_layout(
          title="Mahalanobis Distance and Possible Outliers",
          yaxis_title= "<b>Mahalanobis Distance</b>" ,
          xaxis_title="<b>Tea Samples</b>" ,
          xaxis=dict(
              showgrid=True,
              showline=True,
              showticklabels=False,
              ticks='outside',
              tickcolor='rgb(102, 102, 102)',
              linecolor='rgb(102, 102, 102)',
          ),
          yaxis=dict(
              showgrid=True,
              showline=True,
              showticklabels=True,
              ticks='outside',
              tickcolor='rgb(102, 102, 102)',
              linecolor='rgb(102, 102, 102)',
          ),    
          margin=dict(l=40, r=40, b=40, t=40),
          paper_bgcolor='white',
          plot_bgcolor='white',
          width=600, height=400,
        )
    fig_mahala.show()
    
    isKickOutliers = input('Do you want to kick outliers out? [y/n]')
    
    if isKickOutliers == 'y' or isKickOutliers == 'Y':
        numout=int(input('How many outliers do you want to kick out?'))
        PCR.KickOutliers(X=X, y=y, T=T, mahala=mahala, pc=pc, maxoutliers=numout)
    
    return(fig_mahala)

  def KickOutliers(X, y, T, mahala, pc, maxoutliers):
    # Sort the RMS distance from the origin in descending order (largest first)
    rms_dist = np.sqrt(Q**2+Tsq**2)
    
    # Sort calibration spectra according to descending RMS distance
    X['mahala']=mahala

    Xc=X.sort_values(['mahala'], ascending=False, axis=0)
    Xc=Xc.drop(['mahala'], axis=1)
    X=X.drop(['mahala'], axis=1) 

    mahala=np.flip(np.argsort(mahala), axis=0)
    yc = y[mahala]

    # Discard one outlier at a time up to the value max_outliers
    # and calculate the mse cross-validation of the PLS model
    max_outliers = maxoutliers  
 
    for j in range(max_outliers):
      print('Removed Sample : ' + str(Xc.index[j]))
      PCR.ModelFit(Xc[j:], yc[j:], pc, cv, testsize )


  def ModelFit(X, y, pc, cv, testsize=0.2, figure=True):
    ''' Principal Component Regression in Python'''
    ''' Step 1: PCA on input data'''
    if 'sample' in X.columns:
       start=1
    else:
      start=0

    # Define the PCA object
    pca = PCA()
    

    # Run PCA producing the reduced variable Xred and select the first pc components
    Xreg = pca.fit_transform(X.iloc[:,start:])[:,:pc]
    
    ''' Step 2: regression on selected principal components'''
 
    # Create linear regression object
    regr = linear_model.LinearRegression()
    regr2 = linear_model.LinearRegression()
    # Fit
    regr.fit(Xreg, y)

    x_train, x_test, y_train, y_test = train_test_split(Xreg, y, test_size=testsize, random_state=42) 
    regr2.fit(x_train, y_train)

    # Calibration
    y_c = regr.predict(Xreg)
    # Cross-validation
    if cv == 'loo':
      cv = LeaveOneOut()
    y_cv = cross_val_predict(regr, Xreg, y, cv=cv, n_jobs=-1)
    # Train
    y_v = regr2.predict(x_train)
    # Test
    y_t = regr2.predict(x_test)
 
    # Calculate scores for calibration and cross-validation
    score_c = r2_score(y, y_c)
    score_cv = r2_score(y, y_cv)
    score_t = r2_score(y_test, y_t)
    score_v = r2_score(y_train, y_v)
 
    # Calculate mean square error for calibration and cross validation
    mse_c = np.sqrt(mean_squared_error(y, y_c))
    mse_cv = np.sqrt(mean_squared_error(y, y_cv)) 
    mse_t = np.sqrt(mean_squared_error(y_test, y_t))
    mse_v = np.sqrt(mean_squared_error(y_train, y_v))

    # plot the regression line
    if figure==True:
      z1 = np.polyfit(y, y_c, 1)
      z2 = np.polyfit(y_train, y_v, 1)
      z3 = np.polyfit(y, y_cv, 1)
      PCR_fig = go.Figure()

      PCR_fig.add_trace(go.Scatter(x=y_cv, y=y,
                    mode='markers', marker_symbol='circle',marker_line_width=1,
                    )) #text=X.index, textposition='middle right', name='CV'
      PCR_fig.add_trace(go.Scatter(x=y, y=y,
                    mode='lines', line=dict(color='rgb(102, 102, 102)', width=1, dash='dot')))
      PCR_fig.add_trace(go.Scatter(x=np.polyval(z3,y), y=y,
                    mode='lines', name='Model', marker_color='red'))  
      PCR_fig.add_annotation(text="<i>R<sup>2</sup><sub>CV</sub></i> = "+str(round(score_cv,2)),
                    xref="paper", yref="paper",
                    x=0.05, y=0.99, showarrow=False)
      PCR_fig.add_annotation(text="<i>RMSE<sub>CV</sub></i> = "+str(round(mse_cv,2)),
                    xref="paper", yref="paper",
                    x=0.05, y=0.94, showarrow=False) 
      PCR_fig.add_annotation(text="<i>RPD<sub>CV</sub></i> = "+str(round(1/(mse_cv/statistics.stdev(y)),2)),
                    xref="paper", yref="paper",
                    x=0.05, y=0.89, showarrow=False)
      PCR_fig.add_annotation(text="Slope = "+str(round(1/(np.polyfit(y, y_cv, 1)[0]),2)),
                    xref="paper", yref="paper",
                    x=0.05, y=0.84, showarrow=False)
    
      PCR_fig.update_layout(
        showlegend=False,
        title="Calibration and Cross Validation",
        yaxis_title="<b>Sensorial Appereance Score</b>",
        xaxis_title="<b>Predicted Appereance Score</b>",
        xaxis=dict(
            showgrid=False,
            showline=True,
            showticklabels=True,
            ticks='outside',
            mirror=True,
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
            tick0 = min(y_cv),
            dtick = 2, #75,#2
            title_standoff = 0
        ),
        yaxis=dict(
            showgrid=False,
            showline=True,
            showticklabels=True,
            ticks='outside',
            mirror=True,
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
            tick0 = min(y),
            dtick = 2, #75,#2,
            title_standoff = 0
        ),  
        width=400, height=400,
        margin=dict(l=40, r=40, b=40, t=40),
        paper_bgcolor='white',
        plot_bgcolor='white',
        
      )
      PCR_fig.show()
      plotly.io.write_image(PCR_fig, "H:\sensorfint\Resultsfigure.pdf", format= 'pdf', engine='kaleido')  
    
    return (y_cv, score_c, score_v, score_t, score_cv, mse_c, mse_v, mse_t, mse_cv)#, regr.fit(Xreg, y).coef_, pca

  def ModelCheck(X, y, pc, cv, testsize=0.2, figure=False):
    score=np.empty(shape=(4,pc))
    mse=np.empty(shape=(4,pc))
    for i in range(0,pc,1):
      score[:,i]=PCR.ModelFit(X, y, i+1, cv, testsize, False)[1:5]
      mse[:,i]=PCR.ModelFit(X, y, i+1, cv, testsize, False)[5:]

    score_fig = go.Figure()
    score_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=score[0,:],
                      mode='lines+markers',
                      name='R<sup>2</sup>'))
    score_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=score[1,:],
                      mode='lines+markers',
                      name='R<sup>2</sup>_train'))
    score_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=score[2,:],
                      mode='lines+markers',
                      name='R<sup>2</sup>_test'))
    score_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=score[3,:],
                      mode='lines+markers',
                      name='R<sup>2</sup>_CV'))
  
    mse_fig = go.Figure()
    mse_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=mse[0,:],
                      mode='lines+markers',
                      name='RMSE'))
    mse_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=mse[1,:],
                      mode='lines+markers',
                      name='RMSE_train'))
    mse_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=mse[2,:],
                      mode='lines+markers',
                      name='RMSE_test'))
    mse_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=mse[3,:],
                      mode='lines+markers',
                      name='RMSE_CV')) 
    score_fig.update_layout(
        autosize=False,
        width=500,
        height=500,
        title="R<sup>2</sup> vs. Component Number",
        yaxis_title="R<sup>2</sup>",
        xaxis_title="Number of Components",
        xaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
        ),
        yaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
        ),    
        margin=dict(l=140, r=40, b=50, t=80),
        paper_bgcolor='white',
        plot_bgcolor='white',
      )    
    mse_fig.update_layout(
        autosize=False,
        width=500,
        height=500,
        title="MSE vs. Component Number",
        yaxis_title="MSE",
        xaxis_title="Number of Components",
        xaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
        ),
        yaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
        ),    
        margin=dict(l=140, r=40, b=50, t=80),
        paper_bgcolor='white',
        plot_bgcolor='white',
      )
    score_fig.show()
    mse_fig.show()
    return(list(mse[-1]).index(min(mse[-1]))+1)

  def wlSelection(X, y, pc, cv, testsize=0.2, testcriteria='R2test', figure=False):
    ''' testcriteria: 'r2', 'r2train', 'r2test', r2cv',  'mse', 'msetrain', 'msetest', msecv'
        cv= int or 'llo'   '''
    if 'sample' in X.columns:
       start=1
    else:
      start=0

    '''# Define the PCA object
    pca = PCA()
    
    # Run PCA producing the reduced variable Xred and select the first pc components
    Xreg = pca.fit_transform(X.iloc[:,start:])[:,:pc]

     Step 2: regression on selected principal components
 
    # Create linear regression object
    regr = linear_model.LinearRegression()
    # Fit
    regr.fit(Xreg, y)
    return(regr.coefficients )
    sorted_ind = np.abs(regr.coef_[:,0])

    # Sort calibration spectra according to descending RMS distance
    Xc=X.append(pd.Series(name='sort'))
    Xc.loc['sort']=sorted_ind
    Xc=Xc.sort_values(['sort'], ascending=True, axis=1)
    Xc=Xc.drop(['sort'], axis=0) '''
    
    X_eliminated=X.copy(deep=True)
    if testcriteria=='r2': testnum=1
    elif testcriteria=='r2train': testnum=2
    elif testcriteria=='r2test': testnum=3
    elif testcriteria=='r2cv': testnum=4
    elif testcriteria=='mse': testnum=5
    elif testcriteria=='msetrain': testnum=6
    elif testcriteria=='msetest': testnum=7
    elif testcriteria=='msecv': testnum=8

    matrixlength= X_eliminated.shape[1]-pc
    for j in range(0, X_eliminated.shape[1]-pc):
      if testnum<5:
        if PCR.ModelFit(X_eliminated.drop([str(X.columns[j])], axis=1), y, pc, cv, testsize, figure=False)[testnum] > PCR.ModelFit(X_eliminated.iloc[:,:], y, pc, cv, testsize, figure=False)[testnum]:
          #print('Removed wl : ' + str(X.columns[j]))
          X_eliminated=X_eliminated.drop([str(X.columns[j])], axis=1)
          j -= 1
      else: 
        if PCR.ModelFit(X_eliminated.drop([str(X.columns[j])], axis=1), y, pc, cv, testsize, figure=False)[testnum] < PCR.ModelFit(X_eliminated.iloc[:,:], y, pc, cv, testsize, figure=False)[testnum]:
          #print('Removed wl : ' + str(X.columns[j]))
          X_eliminated=X_eliminated.drop([str(X.columns[j])], axis=1)
          j -= 1
      print(str((j+1)*100/(matrixlength)) + '% completed', end='\r', flush=True)
    
    PCR.ModelFit(X_eliminated.iloc[:,:], y, pc, cv, testsize, figure=True)
    return (X_eliminated, PCR.ModelFit(X_eliminated.iloc[:,:], y, pc, cv, testsize, figure=False)[1:])    
    
    #return (X_eliminated, PCR.ModelFit(X_eliminated.iloc[:,:], y, pc, cv, testsize, figure=True)[1:])
    #return (X_eliminated)
 

class PLS:

  def Outliers(X, y, pc, conf=0.95):
    pls = PLSRegression(n_components=pc)
    # Fit data
    pls.fit(X, y)

    # Get X scores
    T = pls.x_scores_
    # Get X loadings
    P = pls.x_loadings_
    # Calculate error array
    Err = X - np.dot(T,P.T)
    # Calculate Q-residuals (sum over the rows of the error array)
    Q = np.sum(Err**2, axis=1)
    # Calculate Hotelling's T-squared (note that data are normalised by default)
    Tsq = np.sum((pls.x_scores_/np.std(pls.x_scores_, axis=0))**2, axis=1)

    # Calculate confidence level for T-squared from the ppf of the F distribution
    Tsq_conf =  f.ppf(q=conf, dfn=pc, dfd=X.shape[0])*pc*(X.shape[0]-1)/(X.shape[0]-pc)
 
    # Estimate the confidence level for the Q-residuals
    i = np.max(Q) + 1
    while 1-np.sum(Q > i)/np.sum(Q > 0) > conf:
      i -= 1
    Q_conf = i

    #draw the graph
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=Tsq, y=Q,
                      mode='markers+text',marker_symbol='circle',marker_line_width=2, text=Q.index, textposition="top center"))
    fig.add_trace(go.Scatter(x=[Tsq_conf,Tsq_conf], y=[np.min(Q), np.max(Q)],
                      mode='lines', line=dict(color='firebrick', width=2, dash='dash')))
    fig.add_trace(go.Scatter(x=[np.min(Tsq), np.max(Tsq)], y=[Q_conf,Q_conf], 
                      mode='lines', line=dict(color='firebrick', width=2, dash='dash')))
    fig.update(layout_showlegend=False)
    fig.update_layout(
            title="PLS Outliers",
            yaxis_title="Q-residuals",
            xaxis_title="Hotelling’s T<sup>2</sup>",
            xaxis=dict(
                showgrid=True,
                showline=True,
                showticklabels=True,
                ticks='outside',
                tickcolor='rgb(102, 102, 102)',
                linecolor='rgb(102, 102, 102)',
            ),
            yaxis=dict(
                showgrid=True,
                showline=True,
                showticklabels=True,
                ticks='outside',
                tickcolor='rgb(102, 102, 102)',
                linecolor='rgb(102, 102, 102)',
            ),    
            margin=dict(l=140, r=40, b=50, t=80),
            paper_bgcolor='white',
            plot_bgcolor='white',
          )
    fig.show()
    
    numout=int(input('How many outliers do you want to kick out?'))
    PLS.KickOutliers(X=X, y=y, Q=Q, Tsq=Tsq, pc=pc, maxoutliers=numout)

  def KickOutliers(X, y, Q, Tsq, pc, maxoutliers):
    # Sort the RMS distance from the origin in descending order (largest first)
    rms_dist = np.sqrt(Q**2+Tsq**2)
    
    # Sort calibration spectra according to descending RMS distance
    X['rms']=rms_dist
    
    Xc=X.sort_values(['rms'], ascending=False, axis=0)
    Xc=Xc.drop(['rms'], axis=1)
    X=X.drop(['rms'], axis=1) 
       
    rms_dist=np.flip(np.argsort(rms_dist), axis=0)
    yc = y[rms_dist]

    # Discard one outlier at a time up to the value max_outliers
    # and calculate the mse cross-validation of the PLS model
    max_outliers = maxoutliers  


    # Define empty mse array
    mse = np.zeros(max_outliers)
 
    for j in range(max_outliers):
      print('Removed Sample : ' + str(Xc.index[j]))
      PLS.ModelFit(Xc[j:], yc[j:], pc )
 

  def ModelFit(X, y, pc, cv, testsize=0.2, figure=True):
    ''' Partial Least Square Regression'''

    if 'sample' in X.columns:
       start=1
    else:
      start=0

    # Define PLS object 
    pls = PLSRegression(n_components=pc)
    pls2 = PLSRegression(n_components=pc)
    

    # Fit
    pls.fit(X.iloc[:,start:], y)

    #bins = np.linspace(np.floor(np.min(y)), np.ceil(np.max(y)), 5)
    bins = np.array([10, 13, 16, 19])
    y_binned = np.digitize(y, bins)

    x_train, x_test, y_train, y_test = train_test_split(X.iloc[:,start:], y, test_size=testsize, random_state=42) 
    pls2.fit(x_train, y_train)

    # Cross-validation
    if cv == 'loo':
      cv = LeaveOneOut()
    y_cv = cross_val_predict(pls2, X.iloc[:,start:], y, cv=cv, n_jobs=-1)
    y_cv = np.array(y_cv).flatten()
    # Calibration
    y_c = pls.predict(X.iloc[:,start:])
    y_c = np.array(y_c).flatten()
    # Train
    y_v = pls2.predict(x_train)
    y_v = np.array(y_v).flatten()
    # Test
    y_t = pls2.predict(x_test)
    y_t = np.array(y_t).flatten()

    # Calculate scores for calibration and cross-validation
    score_c = r2_score(y, y_c)
    score_cv = r2_score(y, y_cv)
    score_t = r2_score(y_test, y_t)
    score_v = r2_score(y_train, y_v)

    # Calculate mean square error for calibration and cross validation
    mse_c = np.sqrt(mean_squared_error(y, y_c)) 
    mse_cv = np.sqrt(mean_squared_error(y, y_cv))
    mse_t = np.sqrt(mean_squared_error(y_test, y_t))
    mse_v = np.sqrt(mean_squared_error(y_train, y_v))

    # plot the regression line
    if figure==True:
      z1 = np.polyfit(y, y_c, 1)
      z2 = np.polyfit(y_train, y_v, 1)
      z3 = np.polyfit(y, y_cv, 1)
      PCR_fig = go.Figure()
    
      PCR_fig.add_trace(go.Scatter(x=y_cv, y=y,
                    mode='markers', marker_symbol='circle',marker_line_width=1,
                    )) #text=X.index, textposition='middle right', name='CV'
      PCR_fig.add_trace(go.Scatter(x=y, y=y,
                    mode='lines', line=dict(color='rgb(102, 102, 102)', width=1, dash='dot')))

      PCR_fig.add_trace(go.Scatter(x=np.polyval(z3,y), y=y,
                    mode='lines', name='Model', marker_color='red'))
     
      PCR_fig.add_annotation(text="<i>R<sup>2</sup><sub>CV</sub></i> = "+str(round(score_cv,2)),
                    xref="paper", yref="paper",
                    x=0.05, y=0.99, showarrow=False)

      PCR_fig.add_annotation(text="<i>RMSE<sub>CV</sub></i> = "+str(round(mse_cv,2)),
                    xref="paper", yref="paper",
                    x=0.05, y=0.94, showarrow=False) 
      PCR_fig.add_annotation(text="<i>RPD<sub>CV</sub></i> = "+str(round(1/(mse_cv/statistics.stdev(y)),2)),
                    xref="paper", yref="paper",
                    x=0.05, y=0.89, showarrow=False)
      PCR_fig.add_annotation(text="Slope = "+str(round(1/(np.polyfit(y, y_cv, 1)[0]),2)),
                    xref="paper", yref="paper",
                    x=0.05, y=0.84, showarrow=False)
    
      PCR_fig.update_layout(
        showlegend=False,
        title="Calibration and Cross Validation",
        yaxis_title="<b>Measured Moisture Content (%, w/w)</b>",
        xaxis_title="<b>Predicted Moisture Content (%, w/w)</b>",
        xaxis=dict(
            showgrid=False,
            showline=True,
            showticklabels=True,
            ticks='outside',
            mirror=True,
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
            tick0 = min(y_cv),
            dtick = 0.5, #75,#2
            title_standoff = 0
        ),
        yaxis=dict(
            showgrid=False,
            showline=True,
            showticklabels=True,
            ticks='outside',
            mirror=True,
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
            tick0 = min(y),
            dtick = 0.5, #75,#2,
            title_standoff = 0
        ),  
        width=400, height=400,
        margin=dict(l=40, r=40, b=40, t=40),
        paper_bgcolor='white',
        plot_bgcolor='white',
        
      )     
      PCR_fig.show()
      plotly.io.write_image(PCR_fig, "H:\sensorfint\Resultsfigure.pdf", format= 'pdf', engine='kaleido')

    return (y_cv, score_c, score_v, score_t, score_cv, mse_c, mse_v, mse_t, mse_cv)
    

  def ModelCheck(X, y, pc, cv, testsize=0.2, figure=False):
    score=np.empty(shape=(4,pc))
    mse=np.empty(shape=(4,pc))
    for i in range(0,pc,1):
      score[:,i]=PLS.ModelFit(X, y, i+1, cv, testsize, figure)[1:5]

      mse[:,i]=PLS.ModelFit(X, y, i+1, cv, testsize, figure)[5:]

    score_fig = go.Figure()
    score_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=score[0,:],
                      mode='lines+markers',
                      name='R<sup>2</sup>'))
    score_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=score[1,:],
                      mode='lines+markers',
                      name='R<sup>2</sup>_train'))
    score_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=score[2,:],
                      mode='lines+markers',
                      name='R<sup>2</sup>_test'))
    score_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=score[3,:],
                      mode='lines+markers',
                      name='R<sup>2</sup>_CV'))
  
    mse_fig = go.Figure()
    mse_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=mse[0,:],
                      mode='lines+markers',
                      name='MSE'))
    mse_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=mse[1,:],
                      mode='lines+markers',
                      name='MSE_train'))
    mse_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=mse[2,:],
                      mode='lines+markers',
                      name='MSE_test'))
    mse_fig.add_trace(go.Scatter(x=np.arange(1, pc+1), y=mse[3,:],
                      mode='lines+markers',
                      name='MSE_CV'))    

    score_fig.update_layout(
        autosize=False,
        width=500,
        height=500,
        title="R<sup>2</sup> vs. Component Number",
        yaxis_title="R<sup>2</sup>",
        xaxis_title="Number of Components",
        xaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
        ),
        yaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
        ),    
        margin=dict(l=140, r=40, b=50, t=80),
        paper_bgcolor='white',
        plot_bgcolor='white',
      )    
    mse_fig.update_layout(
        autosize=False,
        width=500,
        height=500,
        title="MSE vs. Component Number",
        yaxis_title="MSE",
        xaxis_title="Number of Components",
        xaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
        ),
        yaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            linecolor='rgb(102, 102, 102)',
        ),    
        margin=dict(l=140, r=40, b=50, t=80),
        paper_bgcolor='white',
        plot_bgcolor='white',
      )    
  
    score_fig.show()
    mse_fig.show() 
    return(list(mse[-1]).index(min(mse[-1]))+1)
  
  def wlSelection(X, y, pc, cv, testsize=0.2, testcriteria='r2cv', figure=False):
    ''' testcriteria: 'r2', 'r2train', 'r2test', r2cv',  'mse', 'msetrain', 'msetest', msecv' 
        cv: int or 'llo' '''
    # Get the list of indices that sorts the PLS coefficients in ascending order 
    # of the absolute value
    pls = PLSRegression(n_components=pc)
    # Fit data
    pls.fit(X, y)
    sorted_ind = np.abs(pls.coef_[:,0])

    # Sort calibration spectra according to descending RMS distance
    Xc=X.append(pd.Series(name='sort'))
    Xc.loc['sort']=sorted_ind
    Xc=Xc.sort_values(['sort'], ascending=True, axis=1)
    Xc=Xc.drop(['sort'], axis=0) 
    
    X_eliminated=Xc.copy(deep=True)
    if testcriteria=='r2': testnum=1
    elif testcriteria=='r2train': testnum=2
    elif testcriteria=='r2test': testnum=3
    elif testcriteria=='r2cv': testnum=4
    elif testcriteria=='mse': testnum=5
    elif testcriteria=='msetrain': testnum=6
    elif testcriteria=='msetest': testnum=7
    elif testcriteria=='msecv': testnum=8

    for j in range(0, Xc.shape[1]-pc-1):
        if testnum < 5:
            if PLS.ModelFit(Xc.iloc[:,j+1:], y, pc, cv, testsize, figure=False)[testnum] > PLS.ModelFit(Xc.iloc[:,(j):], y, pc, cv,testsize, figure=False)[testnum]:
                X_eliminated=X_eliminated.drop([str(Xc.columns[j])], axis=1)
        else:
            if PLS.ModelFit(Xc.iloc[:,j+1:], y, pc, cv, testsize, figure=False)[testnum] < PLS.ModelFit(Xc.iloc[:,(j):], y, pc, cv, testsize, figure=False)[testnum]:
                X_eliminated=X_eliminated.drop([str(Xc.columns[j])], axis=1)
        print(str((j+1)*100/(Xc.shape[1]-pc-1)) + '% completed', end='\r', flush=True)  
    
    PLS.ModelFit(X_eliminated.iloc[:,:], y, pc, cv, testsize, figure=True)
    
    return (X_eliminated, PLS.ModelFit(X_eliminated.iloc[:,:], y, pc, cv, testsize, figure=False)[1:])    
        
   

### Data Upload

In [ ]:
os.chdir('H:\sensorfint')
filename='BrukerNewTeaSamples.csv'#'drive/My Drive/peachdata.csv' #'drive/My Drive/repeatability_plate.csv' #'drive/My Drive/BrukerNewTeaSamples.csv'
data=pd.read_csv(filename)

### Raw and Processed Spectra

In [ ]:
data=pd.read_csv(filename)
y=pd.DataFrame(data=data['cellulose']).set_index(data['sample']).astype('float')
y=np.array(y.groupby(y.index)['cellulose'].mean())
data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density',
                'powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
data=data.groupby(['sample']).mean()
dataX=data.iloc[:,:]

dataX.insert(0, 'sample', dataX.index)
X=SmoothandDeriv.Derivative(dataX)

fig1 = DrawSpect(dataX)
fig2 = DrawSpect(X)


In [ ]:
plotly.io.write_image(fig1, "H:\sensorfint\Results/Bruker_raw_forpub.pdf", format= 'pdf', engine='kaleido')
plotly.io.write_image(fig2, "H:\sensorfint\Results/Bruker_1stderiv_frpub.pdf", format= 'pdf', engine='kaleido')

### Repeatability Test

In [ ]:
data=pd.read_csv(filename)
data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density','powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
RMS_repeatability(data, central='mean', conf=0.95, multiplicator=1, reducename=False)

In [ ]:
fig=RMS_graph(RMS_repeatability(data, central='mean', conf=0.95, multiplicator=10, reducename=False))

In [ ]:
plotly.io.write_image(fig, "H:\sensorfint\Results/FOSS_rms_forpub.pdf", format= 'pdf', engine='kaleido')

### Outliers detection - Option 1

In [ ]:
data=pd.read_csv(filename)
y=pd.DataFrame(data=data['cellulose']).set_index(data['sample']).astype('float')
y=pd.array(y.groupby(y.index)['cellulose'].mean())
data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density',
                'powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
data=data.groupby(['sample']).mean()
dataX=data.iloc[:,:] 

dataX.insert(0, 'sample', dataX.index)
X=ScatterCorrection.MSC(SmoothandDeriv.SG(deTrend(dataX, method='highorder', order=2)))

PLS.Outliers(dataX, y, 5, conf=0.9)

### Outliers detection - Option 2

In [ ]:
data=pd.read_csv(filename)
y=pd.DataFrame(data=data['cellulose']).set_index(data['sample']).astype('float')
y=np.array(y.groupby(y.index)['cellulose'].mean())
data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density',
                'powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
data=data.groupby(['sample']).mean()
dataX=data.iloc[:,:] 

dataX.insert(0, 'sample', dataX.index)
#X=ScatterCorrection.MSC(SmoothandDeriv.SG(deTrend(dataX, method='highorder', order=2)))


fig=PCR.Outliers(dataX,y, 5, 'chisq', 0.95)

In [ ]:
plotly.io.write_image(fig, "H:\sensorfint\Results/Bruker_averagedoutliers.pdf", format= 'pdf', engine='kaleido')

### PCR Model Check

In [ ]:
data=pd.read_csv(filename)
y=pd.DataFrame(data=data['cellulose']).set_index(data['sample']).astype('float')
y=np.array(y.groupby(y.index)['cellulose'].mean())
data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density',
                'powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
data=data.groupby(['sample']).mean()
dataX=data.iloc[:,:]

dataX.insert(0, 'sample', dataX.index)
X=SmoothandDeriv.RemoveSpike(ScatterCorrection.MSC(SmoothandDeriv.SG(dataX, window=5)),21)


PCR.ModelCheck(X,y, pc=10, cv='loo', testsize=0.2)

### PLS Model Check

In [ ]:
data=pd.read_csv(filename)
y=pd.DataFrame(data=data['extract']).set_index(data['sample']).astype('float')
y=np.array(y.groupby(y.index)['extract'].mean())

data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density',
                'powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
data=data.groupby(['sample']).mean()
dataX=data.iloc[:,:] 

dataX.insert(0, 'sample', dataX.index)
X=SmoothandDeriv.RemoveSpike(ScatterCorrection.MSC(SmoothandDeriv.SG(dataX, window=5)),21)

PLS.ModelCheck(X,y, pc=10, cv='loo', testsize=0.2)

### PLS Model Fit

In [ ]:
data=pd.read_csv(filename)
y=pd.DataFrame(data=data['cellulose']).set_index(data['sample']).astype('float')
y=np.array(y.groupby(y.index)['cellulose'].mean())
data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density',
                'powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
data=data.groupby(['sample']).mean()
dataX=data.iloc[:,336:-272] 

dataX.insert(0, 'sample', dataX.index)

X=ScatterCorrection.MSC(SmoothandDeriv.SG(dataX, window=11, polyorder=2, derivorder=1))

PLS.ModelFit(X,y,3,cv='loo', testsize=0.2)

### PCR Model Fit

In [ ]:
data=pd.read_csv(filename)
#data=data.loc[data['sample'] != 2020]

y=pd.DataFrame(data=data['cellulose']).set_index(data['sample']).astype('float')

y=pd.array(y.groupby(y.index)['cellulose'].mean())
data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density',
                'powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
data=data.groupby(['sample']).mean()

dataX=data.iloc[:,500:-12] 

dataX.insert(0, 'sample', dataX.index)
X=ScatterCorrection.SNV(SmoothandDeriv.SG(dataX, window=11, polyorder=2, derivorder=1))

PCR.ModelFit(X,y,11, cv='loo', testsize=0.2)

### Optimum Models

In [ ]:
test_variable='cellulose'

data=pd.read_csv(filename)

y=pd.DataFrame(data=data[test_variable]).set_index(data['sample']).astype('float')
y=np.array(y.groupby(y.index)[test_variable].mean())

data=data.drop(['lance' , 'grade', 'colour', 'body', 'quality', 'appereance', 'density',
                'powder', 'moisture', 'extract', 'cellulose' ], axis = 1)
data=data.groupby(['sample']).mean()
dataX=data.iloc[:,475:]          

dataX.insert(0, 'sample', dataX.index)

window=max(round(dataX.shape[1]*0.005) if round(dataX.shape[1]*0.005)%2==1 else round(dataX.shape[1]*0.005)+1, 3)
datas=[dataX,
       ScatterCorrection.MSC(dataX.iloc[:,:], reference='mean'),
       ScatterCorrection.MSC(dataX.iloc[:,:], reference='median'),
       ScatterCorrection.MinMaxNorm(dataX), 
       ScatterCorrection.Normalisation(dataX),
       ScatterCorrection.SNV(dataX),
       SmoothandDeriv.NW(dataX, window=window),
       SmoothandDeriv.SG(dataX, window=window, polyorder=1),
       SmoothandDeriv.SG(dataX, window=window, polyorder=2),
       deTrend(dataX, method='linregres'),
       deTrend(dataX, method='simple'),
       deTrend(dataX, method='highorder', order=2),
       ScatterCorrection.MSC(ScatterCorrection.MSC(dataX, reference='mean'), reference='mean'),
       ScatterCorrection.MSC(ScatterCorrection.MSC(dataX, reference='median'), reference='median'),
       ScatterCorrection.MSC(SmoothandDeriv.SG(dataX, window=window, polyorder=1), reference='mean'),
       ScatterCorrection.MSC(SmoothandDeriv.SG(dataX, window=window, polyorder=2), reference='mean'),
       ScatterCorrection.MSC(SmoothandDeriv.SG(dataX, window=window, polyorder=1), reference='median'),
       ScatterCorrection.MSC(SmoothandDeriv.SG(dataX, window=window, polyorder=2), reference='median'),
       ScatterCorrection.MSC(SmoothandDeriv.NW(dataX, window=window), reference='mean'),
       ScatterCorrection.MSC(SmoothandDeriv.NW(dataX, window=window), reference='median'),
       ScatterCorrection.SNV(SmoothandDeriv.SG(dataX, window=window, polyorder=1)),
       ScatterCorrection.SNV(SmoothandDeriv.SG(dataX, window=window, polyorder=2)),
       ScatterCorrection.SNV(SmoothandDeriv.NW(dataX, window=window)),
       ScatterCorrection.Normalisation(SmoothandDeriv.SG(dataX, window=window, polyorder=1)),
       ScatterCorrection.Normalisation(SmoothandDeriv.SG(dataX, window=window, polyorder=2)),
       ScatterCorrection.Normalisation(SmoothandDeriv.NW(dataX, window=window)),
       ]

In [ ]:
np.savetxt("FOSS_processeddata.txt", 
           datas,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
combinations=['All Data (Noise Removed)', 'MSC_mean', 'MSC_median', 'MinMaxNorm', 'Normalisation', 
                                    'SNV', 'NW', 'SG_order1', 'SG_order2', 'deTrending_linregres', 'deTrending_simple', 
                                    'deTrending_highorder2', 'MSC+MSC_mean', 'MSC+MSC_median', 'MSC_mean+SG_order1', 'MSC_mean+SG_order2', 
                                    'MSC_median+SG_order1', 'MSC_median+SG_order2', 'MSC_mean+NW', 'MSC_median+NW', 'SNV+SG_order1', 
                                    'SNV+SG_order2', 'SNV+NW', 'Normalisation+SG_order1', 'Normalisation+SG_order2', 'Normalisation+NW']
goodnessoffit=pd.DataFrame(columns = ['R2', 'R2_train', 'R2_test', 'R2_CV', 'RMSE', 'RMSE_train', 'RMSE_test', 'RMSE_CV', 'Number of PCs', 'wL'],
                           index = combinations )

In [ ]:
test_variable='moisture'
testsize=0.2
cv='loo'

data=pd.read_csv(filename)

y=pd.DataFrame(data=data[test_variable]).set_index(data['sample']).astype('float')
y=np.array(y.groupby(y.index)[test_variable].mean())

In [ ]:
combinations=['All Data (Noise Removed)', 'MSC_mean', 'MSC_median', 'MinMaxNorm', 'Normalisation', 
                                    'SNV', 'NW', 'SG_order1', 'SG_order2', 'deTrending_linregres', 'deTrending_simple', 
                                    'deTrending_highorder2', 'MSC+MSC_mean', 'MSC+MSC_median', 'MSC_mean+SG_order1', 'MSC_mean+SG_order2', 
                                    'MSC_median+SG_order1', 'MSC_median+SG_order2', 'MSC_mean+NW', 'MSC_median+NW', 'SNV+SG_order1', 
                                    'SNV+SG_order2', 'SNV+NW', 'Normalisation+SG_order1', 'Normalisation+SG_order2', 'Normalisation+NW']
goodnessoffit=pd.DataFrame(columns = ['R2', 'R2_train', 'R2_test', 'R2_CV', 'RMSE', 'RMSE_train', 'RMSE_test', 'RMSE_CV', 'Number of PCs', 'wL'],
                           index = combinations )

regression_method='PLSR'
for goodnessoffit_index in range(7,8):
    if  goodnessoffit_index!=0 and 'FOSS' in filename:
        X=SmoothandDeriv.RemoveSpike(datas[goodnessoffit_index],21)
    else:
        X=datas[goodnessoffit_index]

    stop=False    
    pc=PLS.ModelCheck(X,y, pc=10, cv=cv, testsize=0.2)
    out1=PLS.wlSelection(X,y,pc=pc, cv=cv, testsize=0.2, testcriteria='msecv')
    best_goodness=out1
    counter=3
    while stop==False:
        out2=PLS.wlSelection(out1[0],y,pc=pc, cv=cv, testsize=0.2, testcriteria='msecv')
        if best_goodness[1][7]<=out2[1][7]:
            if counter == 0:
                stop=True
                goodnessoffit.iloc[goodnessoffit_index,:-2]=best_goodness[1:][0]
                goodnessoffit.iloc[goodnessoffit_index,-2]=pc
                goodnessoffit.iloc[goodnessoffit_index,-1]=pd.array(best_goodness[0].columns[1:])
            else:
                counter -= 1
                out1=out2
        else:
            counter==3
            out1=out2
            best_goodness=out2       

In [ ]:
model=PLS.ModelFit(best_goodness[0], y, pc, cv, testsize, figure=True)

In [ ]:
combinations=['All Data (Noise Removed)', 'MSC_mean', 'MSC_median', 'MinMaxNorm', 'Normalisation', 
                                    'SNV', 'NW', 'SG_order1', 'SG_order2', 'deTrending_linregres', 'deTrending_simple', 
                                    'deTrending_highorder2', 'MSC+MSC_mean', 'MSC+MSC_median', 'MSC_mean+SG_order1', 'MSC_mean+SG_order2', 
                                    'MSC_median+SG_order1', 'MSC_median+SG_order2', 'MSC_mean+NW', 'MSC_median+NW', 'SNV+SG_order1', 
                                    'SNV+SG_order2', 'SNV+NW', 'Normalisation+SG_order1', 'Normalisation+SG_order2', 'Normalisation+NW']
goodnessoffit=pd.DataFrame(columns = ['R2', 'R2_train', 'R2_test', 'R2_CV', 'RMSE', 'RMSE_train', 'RMSE_test', 'RMSE_CV', 'Number of PCs', 'wL'],
                           index = combinations )

regression_method='PCR'
for goodnessoffit_index in range(24,25):
    if  goodnessoffit_index!=0 and 'FOSS' in filename:
        X=SmoothandDeriv.RemoveSpike(datas[goodnessoffit_index],21)
    else:
        X=datas[goodnessoffit_index]

    stop=False    
    pc=PCR.ModelCheck(X,y, pc=10, cv=cv, testsize=0.2)
    out1=PCR.wlSelection(X,y,pc=pc, cv=cv, testsize=0.2, testcriteria='msecv')
    best_goodness=out1
    counter=3
    while stop==False:
        out2=PCR.wlSelection(out1[0],y,pc=pc, cv=cv, testsize=0.2, testcriteria='msecv')
        if best_goodness[1][7]<=out2[1][7]:
            if counter == 0:
                stop=True
                goodnessoffit.iloc[goodnessoffit_index,:-2]=best_goodness[1:][0]
                goodnessoffit.iloc[goodnessoffit_index,-2]=pc
                goodnessoffit.iloc[goodnessoffit_index,-1]=pd.array(best_goodness[0].columns[1:])
            else:
                counter -= 1
                out1=out2
        else:
            counter==3
            out1=out2
            best_goodness=out2

In [ ]:
 model=PCR.ModelFit(best_goodness[0], y, pc, cv, testsize, figure=True)

In [ ]:
plotly.io.write_image(fig, "H:\sensorfint\Results/Bruker_averagedoutliers.pdf", format= 'pdf', engine='kaleido')

In [ ]:
model[0]

In [ ]:
model[10].explained_variance_[3]/model[10].explained_variance_ratio[:9].sum()

In [ ]:
min(min(y),min(model[0]))